## Klassifisering av matretter 2

I denne andre leksjonen om klassifisering skal vi utforske `flere måter` å klassifisere kategoriske data på. Vi skal også lære om konsekvensene ved å velge én klassifikator fremfor en annen.

### [**Quiz før forelesning**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Forutsetninger**

Vi antar at du har fullført de tidligere leksjonene, siden vi vil bygge videre på noen konsepter vi lærte tidligere.

For denne leksjonen trenger vi følgende pakker:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling av R-pakker](https://www.tidyverse.org/packages) som er designet for å gjøre dataanalyse raskere, enklere og mer morsomt!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) er et [rammeverk med pakker](https://www.tidymodels.org/packages/) for modellering og maskinlæring.

-   `themis`: [themis-pakken](https://themis.tidymodels.org/) gir ekstra oppskritt for å håndtere ubalanserte data.

Du kan installere dem slik:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Alternativt kan skriptet nedenfor sjekke om du har de nødvendige pakkene for å fullføre dette modulen, og installere dem for deg hvis de mangler.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Et klassifiseringskart**

I vår [forrige leksjon](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) forsøkte vi å besvare spørsmålet: hvordan velger vi mellom flere modeller? I stor grad avhenger det av egenskapene til dataene og typen problem vi ønsker å løse (for eksempel klassifisering eller regresjon).

Tidligere lærte vi om de ulike alternativene du har når du klassifiserer data ved hjelp av Microsofts jukselapp. Python sitt maskinlæringsrammeverk, Scikit-learn, tilbyr en lignende, men mer detaljert jukselapp som kan hjelpe deg med å snevre inn dine estimators (et annet begrep for klassifikatorer):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Tips: [besøk dette kartet online](https://scikit-learn.org/stable/tutorial/machine_learning_map/) og klikk langs stien for å lese dokumentasjonen.
>
> Referansesiden for [Tidymodels](https://www.tidymodels.org/find/parsnip/#models) gir også utmerket dokumentasjon om ulike typer modeller.

### **Planen** 🗺️

Dette kartet er veldig nyttig når du har en klar forståelse av dataene dine, da du kan 'gå' langs stiene til en beslutning:

-   Vi har \>50 prøver

-   Vi ønsker å forutsi en kategori

-   Vi har merket data

-   Vi har færre enn 100K prøver

-   ✨ Vi kan velge en Linear SVC

-   Hvis det ikke fungerer, siden vi har numeriske data

    -   Vi kan prøve en ✨ KNeighbors Classifier

        -   Hvis det ikke fungerer, prøv ✨ SVC og ✨ Ensemble Classifiers

Dette er en veldig nyttig sti å følge. Nå, la oss komme i gang med [tidymodels](https://www.tidymodels.org/) modelleringsrammeverket: en konsistent og fleksibel samling av R-pakker utviklet for å oppmuntre til god statistisk praksis 😊.

## 2. Del opp dataene og håndter ubalanserte datasett.

Fra våre tidligere leksjoner lærte vi at det var et sett med vanlige ingredienser på tvers av våre kjøkken. Det var også en ganske ujevn fordeling i antall kjøkken.

Vi vil håndtere dette ved å

-   Fjerne de vanligste ingrediensene som skaper forvirring mellom ulike kjøkken, ved å bruke `dplyr::select()`.

-   Bruke en `recipe` som forhåndsprosesserer dataene for å gjøre dem klare for modellering ved å bruke en `over-sampling`-algoritme.

Vi har allerede sett på dette i den forrige leksjonen, så dette bør gå som en lek 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Håndtering av ubalanserte data

Ubalanserte data har ofte negative effekter på modellens ytelse. Mange modeller fungerer best når antallet observasjoner er likt, og har derfor en tendens til å slite med ubalanserte data.

Det finnes hovedsakelig to måter å håndtere ubalanserte datasett på:

-   legge til observasjoner i minoritetsklassen: `Over-sampling`, for eksempel ved å bruke en SMOTE-algoritme som syntetisk genererer nye eksempler av minoritetsklassen ved hjelp av nærmeste naboer til disse tilfellene.

-   fjerne observasjoner fra majoritetsklassen: `Under-sampling`

I vår forrige leksjon demonstrerte vi hvordan man kan håndtere ubalanserte datasett ved hjelp av en `recipe`. En recipe kan betraktes som en plan som beskriver hvilke steg som skal brukes på et datasett for å gjøre det klart for dataanalyse. I vårt tilfelle ønsker vi å ha en lik fordeling i antallet av våre matretter for vårt `training set`. La oss sette i gang.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Nå er vi klare til å trene modeller 👩‍💻👨‍💻!

## 3. Utover multinomiale regresjonsmodeller

I vår forrige leksjon så vi på multinomiale regresjonsmodeller. La oss utforske noen mer fleksible modeller for klassifisering.

### Support Vector Machines

I sammenheng med klassifisering er `Support Vector Machines` en maskinlæringsteknikk som prøver å finne et *hyperplan* som "best" skiller klassene. La oss se på et enkelt eksempel:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ skiller ikke klassene. H2~ gjør det, men bare med en liten margin. H3~ skiller dem med maksimal margin.

#### Lineær Support Vector Classifier

Support-Vector clustering (SVC) er en del av Support-Vector-maskiner-familien av ML-teknikker. I SVC velges hyperplanet for å korrekt skille `de fleste` av treningsobservasjonene, men `kan feilkategorisere` noen observasjoner. Ved å tillate at noen punkter er på feil side, blir SVM mer robust mot uteliggere og dermed bedre til å generalisere til nye data. Parameteren som regulerer denne bruddgrensen kalles `cost`, som har en standardverdi på 1 (se `help("svm_poly")`).

La oss lage en lineær SVC ved å sette `degree = 1` i en polynomisk SVM-modell.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Nå som vi har samlet forhåndsprosesseringsstegene og modellspesifikasjonen i en *arbeidsflyt*, kan vi gå videre og trene den lineære SVC og evaluere resultatene samtidig. For ytelsesmålinger, la oss lage et målesett som vil evaluere: `accuracy`, `sensitivity`, `Positive Predicted Value` og `F Measure`.

> `augment()` vil legge til kolonne(r) for prediksjoner til de gitte dataene.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Støttevektormaskin

Støttevektormaskinen (SVM) er en utvidelse av støttevektor-klassifiseringen for å kunne håndtere en ikke-lineær grense mellom klassene. I hovedsak bruker SVM-er *kernel-trikset* for å utvide funksjonsrommet og tilpasse seg ikke-lineære relasjoner mellom klassene. En populær og svært fleksibel kjernefunksjon som brukes av SVM-er er *Radial basis function.* La oss se hvordan den vil prestere på våre data.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Mye bedre 🤩!

> ✅ Vennligst se:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> for videre lesing.

### Klassifisering med nærmeste nabo

*K*-nærmeste nabo (KNN) er en algoritme der hver observasjon blir forutsagt basert på dens *likhet* med andre observasjoner.

La oss tilpasse en til dataene våre.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Det ser ut til at denne modellen ikke presterer så bra. Sannsynligvis vil det å endre modellens argumenter (se `help("nearest_neighbor")`) forbedre modellens ytelse. Sørg for å teste det ut.

> ✅ Vennligst se:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> for å lære mer om *K*-Nearest Neighbors klassifikatorer.

### Ensemble-klassifikatorer

Ensemble-algoritmer fungerer ved å kombinere flere grunnmodeller for å produsere en optimal modell enten ved:

`bagging`: å bruke en *gjennomsnittsfunksjon* på en samling av grunnmodeller

`boosting`: å bygge en sekvens av modeller som bygger på hverandre for å forbedre prediktiv ytelse.

La oss starte med å prøve ut en Random Forest-modell, som bygger en stor samling beslutningstrær og deretter bruker en gjennomsnittsfunksjon for å skape en bedre helhetlig modell.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Godt jobbet 👏!

La oss også eksperimentere med en Boosted Tree-modell.

Boosted Tree definerer en ensemblemetode som lager en serie av sekvensielle beslutningstrær der hvert tre avhenger av resultatene fra tidligere trær i et forsøk på å gradvis redusere feilen. Den fokuserer på vektene til feilklassifiserte elementer og justerer tilpasningen for neste klassifikator for å rette opp.

Det finnes forskjellige måter å tilpasse denne modellen på (se `help("boost_tree")`). I dette eksempelet skal vi tilpasse Boosted trees via `xgboost`-motoren.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Vennligst se:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Utforsker AdaBoost-modellen, som er et godt alternativ til xgboost.
>
> for å lære mer om Ensemble-klassifikatorer.

## 4. Ekstra - sammenligne flere modeller

Vi har tilpasset ganske mange modeller i denne labben 🙌. Det kan bli tidkrevende eller tungvint å lage mange arbeidsflyter fra ulike sett med forbehandlingsmetoder og/eller spesifikasjoner for modeller, og deretter beregne ytelsesmål én etter én.

La oss se om vi kan løse dette ved å lage en funksjon som tilpasser en liste med arbeidsflyter på treningssettet og deretter returnerer ytelsesmål basert på testsettet. Vi skal bruke `map()` og `map_dfr()` fra [purrr](https://purrr.tidyverse.org/) pakken for å anvende funksjoner på hvert element i en liste.

> [`map()`](https://purrr.tidyverse.org/reference/map.html)-funksjoner lar deg erstatte mange for-løkker med kode som både er mer kortfattet og lettere å lese. Det beste stedet å lære om [`map()`](https://purrr.tidyverse.org/reference/map.html)-funksjoner er [iterasjonskapitlet](http://r4ds.had.co.nz/iteration.html) i R for data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) pakken lar brukere lage og enkelt tilpasse et stort antall modeller, men er hovedsakelig designet for å fungere med resamplingsteknikker som `cross-validation`, en tilnærming vi ennå ikke har dekket.

## **🚀Utfordring**

Hver av disse teknikkene har et stort antall parametere som du kan justere, for eksempel `cost` i SVMs, `neighbors` i KNN, `mtry` (Tilfeldig Valgte Prediktorer) i Random Forest.

Undersøk standardparametrene for hver av dem og tenk over hva det å justere disse parameterne vil bety for modellens kvalitet.

For å finne ut mer om en bestemt modell og dens parametere, bruk: `help("model")` f.eks. `help("rand_forest")`

> I praksis *estimerer* vi vanligvis de *beste verdiene* for disse ved å trene mange modeller på et `simulert datasett` og måle hvor godt alle disse modellene presterer. Denne prosessen kalles **tuning**.

### [**Quiz etter forelesning**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Gjennomgang & Selvstudium**

Det er mye fagterminologi i disse leksjonene, så ta et øyeblikk til å gå gjennom [denne listen](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) med nyttige begreper!

#### TAKK TIL:

[`Allison Horst`](https://twitter.com/allison_horst/) for å lage de fantastiske illustrasjonene som gjør R mer innbydende og engasjerende. Finn flere illustrasjoner i hennes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) og [Jen Looper](https://www.twitter.com/jenlooper) for å lage den originale Python-versjonen av dette modulen ♥️

Lykke til med læringen,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Kunstverk av @allison_horst</figcaption>



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
